## Preparar el entorno de trabajo

In [ ]:
# HABILITAR GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importar datos
import pandas as pd
import numpy as np
import glob

# Balancear la muestra
from imblearn.over_sampling import SMOTE

# Evaluar modelos
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Graficar resultados
import plotly.graph_objects as go

## Importar los datos

In [ ]:
# Lista para almacenar los DataFrames individuales
dfs = []

# Ruta a los archivos .csv
file_paths = glob.glob('/content/drive/MyDrive/Ciencia de datos e IA/archive/*.csv')

# Lectura de los .csv como DataFrames e incorporación a la lista
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenamiento de todos los DataFrames a uno solo
global_df = pd.concat(dfs, ignore_index=True)

# Descripción del DataFrame combinado
global_df.describe()

,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label
count,1.832560e+06,1.832560e+06,1.832560e+06,1.832560e+06,1.832560e+06,1.832560e+06,1.832560e+06
mean,-8.683814e-01,-3.178814e-02,2.244210e-02,-6.763959e-01,8.185066e-03,-3.858819e-01,3.940692e+00
std,2.756643e-01,1.556768e-01,4.279549e-01,5.596829e-01,2.707317e-01,5.087015e-01,2.912512e+00
min,-4.333252e+00,-2.031006e+00,-2.204834e+00,-7.942139e+00,-5.142578e+00,-7.593750e+00,1.000000e+00
25%,-9.909670e-01,-1.093750e-01,-2.692870e-01,-9.855960e-01,-1.132810e-01,-9.770510e-01,1.000000e+00
50%,-9.377440e-01,-1.855500e-02,-9.399400e-02,-9.357910e-01,-1.464800e-02,-1.906740e-01,3.000000e+00
75%,-8.344730e-01,5.761700e-02,3.078610e-01,-7.763700e-02,1.230470e-01,-3.174000e-03,7.000000e+00
max,3.630370e-01,1.576660e+00,1.179199e+00,3.395264e+00,5.725098e+00,3.953369e+00,8.000000e+00


In [ ]:
# Crear un DataFrame para cada tipo de actividad
global_df_1 = global_df[(global_df['label'] == 1)]
global_df_3 = global_df[(global_df['label'] == 3)]
global_df_4 = global_df[(global_df['label'] == 4)]
global_df_5 = global_df[(global_df['label'] == 5)]
global_df_6 = global_df[(global_df['label'] == 6)]
global_df_7 = global_df[(global_df['label'] == 7)]
global_df_8 = global_df[(global_df['label'] == 8)]

# Crear un arreglo con todos los DataFrames
global_array = np.array([global_df_1, global_df_3, global_df_4, global_df_5, global_df_6, global_df_7, global_df_8])

<ipython-input-4-ab7247ef5c58>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  global_array = np.array([global_df_1, global_df_3, global_df_4, global_df_5, global_df_6, global_df_7, global_df_8])


## Preparar la muestra de conjuntos de prueba y de entrenamiento

In [ ]:
# Número de muestras por actividad
n_size = 10000

# DataFrame para almacenar la muestra
sample_df = pd.DataFrame()

# Por cada DataFrame de actividades
for df in global_array:

  # Seleccionar todos los datos si es menos que 'n_size'
  if(df.shape[0] < n_size):
    selection = df
  # Seleccionar 'n_size' datos si es mayor que 'n_size'
  else:
    selection = df.sample(n=n_size)

  # Añadir al DataFrame correspondiente
  sample_df = pd.concat([sample_df, selection], axis=0)

# Almacenar la muestra como el DataFrame global
global_df = sample_df
print(global_df['label'].value_counts())

1    10000
3    10000
6    10000
7    10000
8    10000
5     4522
4     3726
Name: label, dtype: int64


In [ ]:
# Identificar variables de prueba y de respuesta
x = global_df.iloc[:, 1:7].to_numpy()
y = global_df.iloc[:, 7].to_numpy()

# Equilibrar la distribución de los datos
smote = SMOTE()
x, y = smote.fit_resample(x, y)

# Número de plieges
n_folds = 5

# Crear divisor de datos para la validación cruzada
kf = StratifiedKFold(n_splits=n_folds, shuffle=True)

# Arreglos para guardar los resultados de cada modelo
precisions = []
recalls = []
f1_scores = []

## Logistic Regression (Lineal)

In [ ]:
# Arreglos para almacenar los valores reales y predicciones
cv_y_test = []
cv_y_pred = []
i = 1

# Repetir el proceso por cada pliege
for train_index, test_index in kf.split(x, y):

    # Imprimir el pliegue actual
    print(f'{i}/{n_folds} FOLDS:', end='')
    i += 1

    # Dividir datos en conjuntos de entrenamiento y de prueba
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(' SPLIT', end='')

    # Crear y entrenar el modelo con el conjunto de entrenamiento
    clf = LogisticRegression(solver='lbfgs', max_iter=1000)
    clf.fit(x_train, y_train)
    print(' - FIT', end='')

    # Probar el modelo con el conjunto de prueba
    y_pred = clf.predict(x_test)
    print(' - PRED', end='')

    # Almacenar los valores verdaderos y predicciones
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
    print(' - SAVE')

# Evaluar el desempeño del modelo
report = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# Arreglos para almacenar los resultados de cada clase
p = []
r = []
f1_s = []

# Por cada clase
for item in report:
  if item.isdigit():
    # Añadir los resultados al arreglo correspondiente
    p.append(report[item]['precision'])
    r.append(report[item]['recall'])
    f1_s.append(report[item]['f1-score'])

# Guardar el arreglo de resultados por modelo
precisions.append(p)
recalls.append(r)
f1_scores.append(f1_s)

1/5 FOLDS: SPLIT - FIT - PRED - SAVE
2/5 FOLDS: SPLIT - FIT - PRED - SAVE
3/5 FOLDS: SPLIT - FIT - PRED - SAVE
4/5 FOLDS: SPLIT - FIT - PRED - SAVE
5/5 FOLDS: SPLIT - FIT - PRED - SAVE
              precision    recall  f1-score   support

           1       0.27      0.21      0.23     10000
           3       0.31      0.20      0.24     10000
           4       0.46      0.62      0.53     10000
           5       0.41      0.31      0.35     10000
           6       0.39      0.57      0.46     10000
           7       0.96      0.97      0.96     10000
           8       0.98      0.98      0.98     10000

    accuracy                           0.55     70000
   macro avg       0.54      0.55      0.54     70000
weighted avg       0.54      0.55      0.54     70000



## Linear SVM (Lineal)

In [ ]:
# Arreglos para almacenar los valores reales y predicciones
cv_y_test = []
cv_y_pred = []
i = 1

# Repetir el proceso por cada pliege
for train_index, test_index in kf.split(x, y):

    # Imprimir el pliegue actual
    print(f'{i}/{n_folds} FOLDS:', end='')
    i += 1

    # Dividir datos en conjuntos de entrenamiento y de prueba
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(' SPLIT', end='')

    # Crear y entrenar el modelo con el conjunto de entrenamiento
    clf = SVC(kernel='linear')
    clf.fit(x_train, y_train)
    print(' - FIT', end='')

    # Probar el modelo con el conjunto de prueba
    y_pred = clf.predict(x_test)
    print(' - PRED', end='')

    # Almacenar los valores verdaderos y predicciones
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
    print(' - SAVE')

# Evaluar el desempeño del modelo
report = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# Arreglos para almacenar los resultados de cada clase
p = []
r = []
f1_s = []

# Por cada clase
for item in report:
  if item.isdigit():
    # Añadir los resultados al arreglo correspondiente
    p.append(report[item]['precision'])
    r.append(report[item]['recall'])
    f1_s.append(report[item]['f1-score'])

# Guardar el arreglo de resultados por modelo
precisions.append(p)
recalls.append(r)
f1_scores.append(f1_s)

1/5 FOLDS: SPLIT - FIT - PRED - SAVE
2/5 FOLDS: SPLIT - FIT - PRED - SAVE
3/5 FOLDS: SPLIT - FIT - PRED - SAVE
4/5 FOLDS: SPLIT - FIT - PRED - SAVE
5/5 FOLDS: SPLIT - FIT - PRED - SAVE
              precision    recall  f1-score   support

           1       0.32      0.22      0.26     10000
           3       0.30      0.26      0.28     10000
           4       0.50      0.60      0.54     10000
           5       0.44      0.37      0.41     10000
           6       0.40      0.55      0.46     10000
           7       0.97      0.98      0.97     10000
           8       0.99      0.98      0.98     10000

    accuracy                           0.57     70000
   macro avg       0.56      0.57      0.56     70000
weighted avg       0.56      0.57      0.56     70000



## RBF/Radial SVM (No lineal)

In [ ]:
# Arreglos para almacenar los valores reales y predicciones
cv_y_test = []
cv_y_pred = []
i = 1

# Repetir el proceso por cada pliege
for train_index, test_index in kf.split(x, y):

    # Imprimir el pliegue actual
    print(f'{i}/{n_folds} FOLDS:', end='')
    i += 1

    # Dividir datos en conjuntos de entrenamiento y de prueba
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(' SPLIT', end='')

    # Crear y entrenar el modelo con el conjunto de entrenamiento
    clf = SVC(kernel='rbf')
    clf.fit(x_train, y_train)
    print(' - FIT', end='')

    # Probar el modelo con el conjunto de prueba
    y_pred = clf.predict(x_test)
    print(' - PRED', end='')

    # Almacenar los valores verdaderos y predicciones
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
    print(' - SAVE')

# Evaluar el desempeño del modelo
report = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# Arreglos para almacenar los resultados de cada clase
p = []
r = []
f1_s = []

# Por cada clase
for item in report:
  if item.isdigit():
    # Añadir los resultados al arreglo correspondiente
    p.append(report[item]['precision'])
    r.append(report[item]['recall'])
    f1_s.append(report[item]['f1-score'])

# Guardar el arreglo de resultados por modelo
precisions.append(p)
recalls.append(r)
f1_scores.append(f1_s)

1/5 FOLDS: SPLIT - FIT - PRED - SAVE
2/5 FOLDS: SPLIT - FIT - PRED - SAVE
3/5 FOLDS: SPLIT - FIT - PRED - SAVE
4/5 FOLDS: SPLIT - FIT - PRED - SAVE
5/5 FOLDS: SPLIT - FIT - PRED - SAVE
              precision    recall  f1-score   support

           1       0.67      0.43      0.52     10000
           3       0.39      0.43      0.41     10000
           4       0.65      0.57      0.61     10000
           5       0.56      0.52      0.54     10000
           6       0.54      0.78      0.64     10000
           7       0.99      0.98      0.98     10000
           8       0.99      0.99      0.99     10000

    accuracy                           0.67     70000
   macro avg       0.68      0.67      0.67     70000
weighted avg       0.68      0.67      0.67     70000



## Random Forest (No lineal)

In [ ]:
# Arreglos para almacenar los valores reales y predicciones
cv_y_test = []
cv_y_pred = []
i = 1

# Repetir el proceso por cada pliege
for train_index, test_index in kf.split(x, y):

    # Imprimir el pliegue actual
    print(f'{i}/{n_folds} FOLDS:', end='')
    i += 1

    # Dividir datos en conjuntos de entrenamiento y de prueba
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(' SPLIT', end='')

    # Crear y entrenar el modelo con el conjunto de entrenamiento
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    print(' - FIT', end='')

    # Probar el modelo con el conjunto de prueba
    y_pred = clf.predict(x_test)
    print(' - PRED', end='')

    # Almacenar los valores verdaderos y predicciones
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
    print(' - SAVE')

# Evaluar el desempeño del modelo
report = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# Arreglos para almacenar los resultados de cada clase
p = []
r = []
f1_s = []

# Por cada clase
for item in report:
  if item.isdigit():
    # Añadir los resultados al arreglo correspondiente
    p.append(report[item]['precision'])
    r.append(report[item]['recall'])
    f1_s.append(report[item]['f1-score'])

# Guardar el arreglo de resultados por modelo
precisions.append(p)
recalls.append(r)
f1_scores.append(f1_s)

1/5 FOLDS: SPLIT - FIT - PRED - SAVE
2/5 FOLDS: SPLIT - FIT - PRED - SAVE
3/5 FOLDS: SPLIT - FIT - PRED - SAVE
4/5 FOLDS: SPLIT - FIT - PRED - SAVE
5/5 FOLDS: SPLIT - FIT - PRED - SAVE
              precision    recall  f1-score   support

           1       0.75      0.68      0.71     10000
           3       0.64      0.66      0.65     10000
           4       0.86      0.91      0.88     10000
           5       0.83      0.84      0.83     10000
           6       0.82      0.82      0.82     10000
           7       1.00      1.00      1.00     10000
           8       1.00      1.00      1.00     10000

    accuracy                           0.84     70000
   macro avg       0.84      0.84      0.84     70000
weighted avg       0.84      0.84      0.84     70000



## Gradient Boosting (No lineal)

In [ ]:
# Arreglos para almacenar los valores reales y predicciones
cv_y_test = []
cv_y_pred = []
i = 1

# Repetir el proceso por cada pliege
for train_index, test_index in kf.split(x, y):

    # Imprimir el pliegue actual
    print(f'{i}/{n_folds} FOLDS:', end='')
    i += 1

    # Dividir datos en conjuntos de entrenamiento y de prueba
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(' SPLIT', end='')

    # Crear y entrenar el modelo con el conjunto de entrenamiento
    clf = GradientBoostingClassifier()
    clf.fit(x_train, y_train)
    print(' - FIT', end='')

    # Probar el modelo con el conjunto de prueba
    y_pred = clf.predict(x_test)
    print(' - PRED', end='')

    # Almacenar los valores verdaderos y predicciones
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
    print(' - SAVE')

# Evaluar el desempeño del modelo
report = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# Arreglos para almacenar los resultados de cada clase
p = []
r = []
f1_s = []

# Por cada clase
for item in report:
  if item.isdigit():
    # Añadir los resultados al arreglo correspondiente
    p.append(report[item]['precision'])
    r.append(report[item]['recall'])
    f1_s.append(report[item]['f1-score'])

# Guardar el arreglo de resultados por modelo
precisions.append(p)
recalls.append(r)
f1_scores.append(f1_s)

1/5 FOLDS: SPLIT - FIT - PRED - SAVE
2/5 FOLDS: SPLIT - FIT - PRED - SAVE
3/5 FOLDS: SPLIT - FIT - PRED - SAVE
4/5 FOLDS: SPLIT - FIT - PRED - SAVE
5/5 FOLDS: SPLIT - FIT - PRED - SAVE
              precision    recall  f1-score   support

           1       0.63      0.54      0.58     10000
           3       0.50      0.54      0.52     10000
           4       0.67      0.69      0.68     10000
           5       0.59      0.54      0.56     10000
           6       0.70      0.81      0.75     10000
           7       0.99      0.99      0.99     10000
           8       1.00      1.00      1.00     10000

    accuracy                           0.73     70000
   macro avg       0.73      0.73      0.73     70000
weighted avg       0.73      0.73      0.73     70000



## K-Nearest Neighbor (No lineal)

In [ ]:
# Arreglos para almacenar los valores reales y predicciones
cv_y_test = []
cv_y_pred = []
i = 1

# Repetir el proceso por cada pliege
for train_index, test_index in kf.split(x, y):

    # Imprimir el pliegue actual
    print(f'{i}/{n_folds} FOLDS:', end='')
    i += 1

    # Dividir datos en conjuntos de entrenamiento y de prueba
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(' SPLIT', end='')

    # Crear y entrenar el modelo con el conjunto de entrenamiento
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train)
    print(' - FIT', end='')

    # Probar el modelo con el conjunto de prueba
    y_pred = clf.predict(x_test)
    print(' - PRED', end='')

    # Almacenar los valores verdaderos y predicciones
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
    print(' - SAVE')

# Evaluar el desempeño del modelo
report = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# Arreglos para almacenar los resultados de cada clase
p = []
r = []
f1_s = []

# Por cada clase
for item in report:
  if item.isdigit():
    # Añadir los resultados al arreglo correspondiente
    p.append(report[item]['precision'])
    r.append(report[item]['recall'])
    f1_s.append(report[item]['f1-score'])

# Guardar el arreglo de resultados por modelo
precisions.append(p)
recalls.append(r)
f1_scores.append(f1_s)

1/5 FOLDS: SPLIT - FIT - PRED - SAVE
2/5 FOLDS: SPLIT - FIT - PRED - SAVE
3/5 FOLDS: SPLIT - FIT - PRED - SAVE
4/5 FOLDS: SPLIT - FIT - PRED - SAVE
5/5 FOLDS: SPLIT - FIT - PRED - SAVE
              precision    recall  f1-score   support

           1       0.74      0.61      0.67     10000
           3       0.63      0.58      0.60     10000
           4       0.82      0.92      0.86     10000
           5       0.78      0.81      0.80     10000
           6       0.77      0.83      0.80     10000
           7       1.00      1.00      1.00     10000
           8       1.00      1.00      1.00     10000

    accuracy                           0.82     70000
   macro avg       0.82      0.82      0.82     70000
weighted avg       0.82      0.82      0.82     70000



## Decision Tree (No lineal)

In [ ]:
# Arreglos para almacenar los valores reales y predicciones
cv_y_test = []
cv_y_pred = []
i = 1

# Repetir el proceso por cada pliege
for train_index, test_index in kf.split(x, y):

    # Imprimir el pliegue actual
    print(f'{i}/{n_folds} FOLDS:', end='')
    i += 1

    # Dividir datos en conjuntos de entrenamiento y de prueba
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(' SPLIT', end='')

    # Crear y entrenar el modelo con el conjunto de entrenamiento
    clf = DecisionTreeClassifier()
    clf.fit(x_train, y_train)
    print(' - FIT', end='')

    # Probar el modelo con el conjunto de prueba
    y_pred = clf.predict(x_test)
    print(' - PRED', end='')

    # Almacenar los valores verdaderos y predicciones
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
    print(' - SAVE')

# Evaluar el desempeño del modelo
report = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# Arreglos para almacenar los resultados de cada clase
p = []
r = []
f1_s = []

# Por cada clase
for item in report:
  if item.isdigit():
    # Añadir los resultados al arreglo correspondiente
    p.append(report[item]['precision'])
    r.append(report[item]['recall'])
    f1_s.append(report[item]['f1-score'])

# Guardar el arreglo de resultados por modelo
precisions.append(p)
recalls.append(r)
f1_scores.append(f1_s)

1/5 FOLDS: SPLIT - FIT - PRED - SAVE
2/5 FOLDS: SPLIT - FIT - PRED - SAVE
3/5 FOLDS: SPLIT - FIT - PRED - SAVE
4/5 FOLDS: SPLIT - FIT - PRED - SAVE
5/5 FOLDS: SPLIT - FIT - PRED - SAVE
              precision    recall  f1-score   support

           1       0.60      0.57      0.58     10000
           3       0.52      0.50      0.51     10000
           4       0.78      0.81      0.79     10000
           5       0.70      0.72      0.71     10000
           6       0.73      0.74      0.73     10000
           7       0.99      0.99      0.99     10000
           8       1.00      1.00      1.00     10000

    accuracy                           0.76     70000
   macro avg       0.76      0.76      0.76     70000
weighted avg       0.76      0.76      0.76     70000



## Gráficos

In [ ]:
# Clases
categories = ['1', '3', '4', '5', '6', '7', '8']

# Modelos
labels = ['Logistic Regression', 'Linear SVM', 'RBF/Radial SVM', 'Random Forest', 'Gradient Boosting', 'K-Nearest Neighbor', 'Decision Tree']


In [ ]:
# Arreglo para los gráficos
radar_traces = []

# Para cada modelo
for i, data in enumerate(precisions):

    # Graficar los datos correspondientes
    trace = go.Scatterpolar(
        r=data,
        theta=categories,
        name=labels[i]
    )

    # Añadir al arreglo correspondiente
    radar_traces.append(trace)

# Crear un diseño para los gráficos
layout = go.Layout(
    title='Precision by Class',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True
)

# Ligar los gráficos con el modelo
fig = go.Figure(data=radar_traces, layout=layout)

# Mostrar la figura
fig.show()

In [ ]:
# Arreglo para los gráficos
radar_traces = []

# Para cada modelo
for i, data in enumerate(recalls):

    # Graficar los datos correspondientes
    trace = go.Scatterpolar(
        r=data,
        theta=categories,
        name=labels[i]
    )

    # Añadir al arreglo correspondiente
    radar_traces.append(trace)

# Crear un diseño para los gráficos
layout = go.Layout(
    title='Recall by Class',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True
)

# Ligar los gráficos con el modelo
fig = go.Figure(data=radar_traces, layout=layout)

# Mostrar la figura
fig.show()

In [ ]:
# Arreglo para los gráficos
radar_traces = []

# Para cada modelo
for i, data in enumerate(f1_scores):

    # Graficar los datos correspondientes
    trace = go.Scatterpolar(
        r=data,
        theta=categories,
        name=labels[i]
    )

    # Añadir al arreglo correspondiente
    radar_traces.append(trace)

# Crear un diseño para los gráficos
layout = go.Layout(
    title='F1 Scores by Class',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True
)

# Ligar los gráficos con el modelo
fig = go.Figure(data=radar_traces, layout=layout)

# Mostrar la figura
fig.show()